# Create Apects and Embedding for every Review
* using all_df_filtered dataset

# Import

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle


In [3]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
RANDOM_STATE = 24

In [20]:
# 導入資料
all_df = pd.read_pickle('../Data/restaurant_only_filtered.pkl').reset_index(drop=True)
lda_model = pickle.load(open('../data/lda_model.pkl', 'rb'))
docs = pickle.load(open("../data/LDA_training_docs.pkl", "rb"))

In [15]:
print(all_df.shape)
print(f'Number of unique tokens: {len(dictionary)}')

(387723, 20)
Number of unique tokens: 1000


In [21]:
all_df.head()

,business_id,stars_x,useful,funny,cool,text,date,name,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,Underwhelming. Food is ok. Service is stand...,2021-05-17 12:47:05,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
1,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,Enjoyed a solid brunch here Saturday. Has bot...,2021-10-11 02:05:12,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
2,XQfwVwDr-v0ZS3_CbbE5Xw,5.0,0,0,0,A big shout out to the servers and management ...,2021-09-22 13:06:13,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
3,XQfwVwDr-v0ZS3_CbbE5Xw,5.0,0,0,0,Best brunch menu around. Something for everyon...,2021-04-18 23:34:04,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
4,XQfwVwDr-v0ZS3_CbbE5Xw,1.0,0,0,0,"Terrible host service, 45 minute wait, seated ...",2021-09-11 15:08:33,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."


# Review Features

## General Inquirer

In [ ]:
import nltk
from nltk.corpus import opinion_lexicon

# Download the General Inquirer lexicon
nltk.download('opinion_lexicon')
general_inquirer_words = set(opinion_lexicon.words())

# Function to check if a string contains General Inquirer words
def contains_general_inquirer(text):
    words = set(text.lower().split())
    common_words = words.intersection(general_inquirer_words)
    return len(common_words) > 0

In [ ]:
# # unhash if necessary
# cnt = 0
# for idx , row in all_df.iterrows():
#     # if cnt%10000 == 0:
#     #     print(f'Now progress .... {cnt}')
#     if contains_general_inquirer(row.text):
#         pass
#     else:
#         cnt+=1
#         row.text = ''
# all_df.to_pickle('../Data/restaurant_only_filtered.pkl')

## LDA 

In [16]:
import spacy
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim import matutils
#python -m spacy download en_core_web_md 

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

In [17]:
nlp = spacy.load("en_core_web_md")
keep_pos = ['NOUN','ADJ','ADV','VERB']
# removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']

def preprocess(text : str) -> list:
    
    for summary in nlp.pipe([text]):
        proj_tok = [token.lemma_.lower() for token in summary \
                    if token.pos_ in keep_pos and not token.is_stop and token.is_alpha]

    return proj_tok

In [ ]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=50, no_above=0.5,keep_n=1000)

In [ ]:
# check topic
lda_model.print_topics(-1)

In [ ]:
# # unhash if necessary 
# def get_training_docs(all_df):
    
#     docs = []
    
#     for name in Counter(all_df.name):
#         res = all_df[all_df.name == name]
        
#         docs.append(preprocess(''.join(res.text.values.tolist())[:5000]))

#     return docs

# docs = get_training_docs(all_df)
# with open("../data/LDA_training_docs.pkl","wb") as file:
#     # Use pickle.dump() to serialize and save the object to the file
#     pickle.dump(docs, file)

In [ ]:
# Train LDA
# #### doc2Bow
# corpus = [dictionary.doc2bow(doc) for doc in docs]

# # # building models
# lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, \
#                          num_topics=50, workers = 4, passes=10)

# with open('../data/lda_model.pkl', 'wb') as file:
#     # Use pickle.dump() to serialize and save the object to the file
#     pickle.dump(lda_model, file)

## Bert Embedding

In [22]:
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name,output_hidden_states = True).to(device)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [51]:
# get LDA aspects 
def get_aspect(text):
    corpus = [dictionary.doc2bow(doc) for doc in [preprocess(text)]]
    return lda_model[corpus][0]

def get_bert_embedding(text):
    encoded_input = tokenizer( text, max_length = 128 , padding = True , truncation = True,  return_tensors='pt')
    encoded_input = encoded_input.to(device)
    with torch.no_grad():
        outputs = model(**encoded_input)
    
    return outputs.hidden_states[-1][0,0,:].to('cpu')

In [26]:
all_df['LDA_aspects'] = all_df['text'].apply(lambda x : get_aspect(x))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [52]:
all_df['BERT_embedding'] = all_df['text'].apply(lambda x : get_bert_embedding(x))

In [53]:
all_df.head()

,business_id,stars_x,useful,funny,cool,text,date,name,address,city,...,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours,LDA_aspects,BERT_embedding
0,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,Underwhelming. Food is ok. Service is stand...,2021-05-17 12:47:05,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,...,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'...","[(10, 0.14239219), (20, 0.40559912), (48, 0.38...","[tensor(0.0710), tensor(0.0343), tensor(0.6646..."
1,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,Enjoyed a solid brunch here Saturday. Has bot...,2021-10-11 02:05:12,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,...,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'...","[(48, 0.8774869)]","[tensor(0.0155), tensor(-0.0770), tensor(0.313..."
2,XQfwVwDr-v0ZS3_CbbE5Xw,5.0,0,0,0,A big shout out to the servers and management ...,2021-09-22 13:06:13,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,...,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'...","[(6, 0.49286205), (18, 0.41985303)]","[tensor(-0.2044), tensor(0.0056), tensor(-0.02..."
3,XQfwVwDr-v0ZS3_CbbE5Xw,5.0,0,0,0,Best brunch menu around. Something for everyon...,2021-04-18 23:34:04,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,...,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'...","[(20, 0.8366538)]","[tensor(-0.4141), tensor(-0.0847), tensor(-0.0..."
4,XQfwVwDr-v0ZS3_CbbE5Xw,1.0,0,0,0,"Terrible host service, 45 minute wait, seated ...",2021-09-11 15:08:33,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,...,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': ""u'average'"", 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'...","[(41, 0.9346619)]","[tensor(-0.5058), tensor(0.0709), tensor(0.393..."


In [54]:
all_df.to_pickle('../Data/restaurant_only_filtered_with_embedding.pkl')